In [ ]:
# Clone repository
%cd /content
!rm -rf Deep_Learning_Project_Gil_Alon
!git clone https://github.com/gil-attar/Deep_Learning_Project_Gil_Alon.git
%cd Deep_Learning_Project_Gil_Alon

In [ ]:
# Install dependencies
!pip install roboflow pyyaml pillow -q

In [ ]:
import os
os.environ["ROBOFLOW_API_KEY"] = "zEF9icmDY2oTcPkaDcQY"

# Download dataset
!python scripts/download_dataset.py --output_dir data/raw

# Build evaluation index (creates test_index.json)
!python scripts/build_evaluation_index.py --dataset_root data/raw --output_dir data/processed

In [ ]:
# Generate synthetic occlusions
!python scripts/generate_synthetic_occlusions.py \
    --test_index data/processed/evaluation/test_index.json \
    --images_dir data/raw/test/images \
    --labels_dir data/raw/test/labels \
    --output_dir data/synthetic_occlusion \
    --levels 0.2,0.4,0.6 \
    --seed 42

In [ ]:
# Verify output structure
!echo "=== Output Structure ==="
!ls -la data/synthetic_occlusion/
!echo ""
!echo "=== Level 020 (20% occlusion) ==="
!ls data/synthetic_occlusion/level_020/images/ | head -5
!echo ""
!echo "=== Image counts ==="
!echo "Level 020: $(ls data/synthetic_occlusion/level_020/images/ | wc -l) images"
!echo "Level 040: $(ls data/synthetic_occlusion/level_040/images/ | wc -l) images"
!echo "Level 060: $(ls data/synthetic_occlusion/level_060/images/ | wc -l) images"

## Visualize 10 Examples Per Level

Compare original vs 20% vs 40% vs 60% occlusion for the same images.

In [ ]:
import matplotlib.pyplot as plt
from PIL import Image
from pathlib import Path
import random

# Get 10 random test images
original_dir = Path('data/raw/test/images')
all_images = sorted(original_dir.glob('*.jpg'))

random.seed(42)
sample_images = random.sample(all_images, min(10, len(all_images)))

print(f"Selected {len(sample_images)} images for visualization")

In [ ]:
# Visualize: 10 rows x 4 columns (original, 20%, 40%, 60%)
fig, axes = plt.subplots(10, 4, figsize=(16, 40))

levels = ['original', 'level_020', 'level_040', 'level_060']
titles = ['Original', '20% Occluded', '40% Occluded', '60% Occluded']

for row, img_path in enumerate(sample_images):
    filename = img_path.name
    
    for col, (level, title) in enumerate(zip(levels, titles)):
        if level == 'original':
            path = img_path
        else:
            path = Path(f'data/synthetic_occlusion/{level}/images/{filename}')
        
        if path.exists():
            img = Image.open(path)
            axes[row, col].imshow(img)
        else:
            axes[row, col].text(0.5, 0.5, 'Not found', ha='center', va='center')
        
        if row == 0:
            axes[row, col].set_title(title, fontsize=14, fontweight='bold')
        
        axes[row, col].axis('off')

plt.tight_layout()
plt.savefig('evaluation/synthetic_occlusion_examples.png', dpi=150, bbox_inches='tight')
plt.show()

print("\n✓ Saved to evaluation/synthetic_occlusion_examples.png")

In [ ]:
# Detailed view: Show bounding boxes on one example
import json
from PIL import ImageDraw

# Load test index to get bounding boxes
with open('data/processed/evaluation/test_index.json', 'r') as f:
    test_index = json.load(f)

# Pick first sample image
sample_filename = sample_images[0].name
sample_id = sample_images[0].stem

# Find this image in the index
img_data = None
for img in test_index['images']:
    if img['image_filename'] == sample_filename:
        img_data = img
        break

if img_data:
    fig, axes = plt.subplots(1, 4, figsize=(20, 5))
    
    for col, (level, title) in enumerate(zip(levels, titles)):
        if level == 'original':
            path = sample_images[0]
        else:
            path = Path(f'data/synthetic_occlusion/{level}/images/{sample_filename}')
        
        img = Image.open(path).copy()
        draw = ImageDraw.Draw(img)
        
        # Draw bounding boxes in green
        for box in img_data['ground_truth']:
            bbox = box['bbox_xyxy']
            draw.rectangle(bbox, outline='lime', width=3)
            draw.text((bbox[0], bbox[1]-15), box['class_name'], fill='lime')
        
        axes[col].imshow(img)
        axes[col].set_title(title, fontsize=14, fontweight='bold')
        axes[col].axis('off')
    
    plt.suptitle(f'Example: {sample_filename}\nGreen boxes = Ground Truth', fontsize=16)
    plt.tight_layout()
    plt.savefig('evaluation/occlusion_with_boxes.png', dpi=150, bbox_inches='tight')
    plt.show()
    
    print(f"\nImage: {sample_filename}")
    print(f"Difficulty: {img_data['difficulty']}")
    print(f"Objects: {img_data['num_objects']}")
    print(f"Classes: {[b['class_name'] for b in img_data['ground_truth']]}")
else:
    print(f"Image {sample_filename} not found in index")

In [ ]:
print("=" * 60)
print("SYNTHETIC OCCLUSION TEST COMPLETE")
print("=" * 60)
print("")
print("If the visualizations look correct:")
print("  - Black rectangles cover parts of ingredients")
print("  - 20% < 40% < 60% in coverage")
print("  - Ground truth boxes still visible (green)")
print("")
print("Then the script is working! Ready to commit.")